In [1]:
# Notebook to diagnose in order to be able to log compute metrics

In [2]:
from functools import partial
import pickle

import evaluate
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, AutoConfig

c:\Users\Roma\.virtualenvs\code_gen-n2ugPVn-\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


c:\Users\Roma\.virtualenvs\code_gen-n2ugPVn-\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [3]:
def tokenize_function(example, tokenizer, seq_length):
    return tokenizer(
        example['text'],
        padding="max_length",
        truncation=True,
        max_length=seq_length,
    )

In [4]:
def add_labels(example):
    example['label'] = example['input_ids']
    return example

In [5]:
# Data saved in create_dataset_save.py module
with open("../data/dataset_hf_train_val.pkl", "rb") as f:
    # Read the data from the file
    data_prepped = pickle.load(f)

In [6]:
checkpoint = "Salesforce/codegen-350M-mono"
device = "cpu"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True).to(device)

In [7]:
def compute_chrf_score(preds, tokenizer, metric):
    logits = preds.predictions[0]
    preds_tok = np.argmax(logits, axis=1)
    acts = preds.label_ids

    decode_predictions = tokenizer.batch_decode(preds_tok,
                                                skip_special_tokens=True)
    decode_labels = tokenizer.batch_decode(acts, skip_special_tokens=True)

    res = metric.compute(predictions=decode_predictions,
                         references=decode_labels)
    return {'chrf_score': res['score']}

In [8]:
seq_length = 40
batch_size = 2

In [9]:
tokenize_partial = partial(tokenize_function,
                               tokenizer=tokenizer,
                               seq_length=seq_length)

tokenized_dataset = data_prepped.map(tokenize_partial,
                                batched=True,
                                batch_size=batch_size).map(
                                    add_labels,
                                    batched=True,
                                    batch_size=batch_size)

Map:   0%|          | 0/157 [00:00<?, ? examples/s]

Map: 100%|██████████| 53/53 [00:00<00:00, 3312.25 examples/s]


In [10]:
chrf = evaluate.load("chrf")
compute_metric_partial = partial(compute_chrf_score, tokenizer = tokenizer, metric = chrf)

In [11]:
# Data collator - Assembles data into batches for training
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(output_dir='../trainer_res',
                                  gradient_checkpointing=True,
                                  evaluation_strategy="epoch",
                                  num_train_epochs=3)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metric_partial,
    tokenizer=tokenizer
)

trainer.train()

Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (packaging 23.1 (c:\users\roma\.virtualenvs\code_gen-n2ugpvn-\lib\site-packages), Requirement.parse('packaging<=23.0,>=20.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (packaging 23.1 (c:\users\roma\.virtualenvs\code_gen-n2ugpvn-\lib\site-packages), Requirement.parse('packaging<=23.0,>=20.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (packaging 23.1 (c:\users\roma\.virtualenvs\code_gen-n2ugpvn-\lib\site-packages), Requirement.parse('packaging<=23.0,>=20.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = 

{'eval_loss': 2.313001871109009, 'eval_chrf_score': 0.029652109121347678, 'eval_runtime': 39.6936, 'eval_samples_per_second': 1.335, 'eval_steps_per_second': 0.176, 'epoch': 1.0}


                                               
 67%|██████▋   | 40/60 [06:55<02:09,  6.50s/it]

{'eval_loss': 2.269188165664673, 'eval_chrf_score': 0.028982331097937207, 'eval_runtime': 98.8618, 'eval_samples_per_second': 0.536, 'eval_steps_per_second': 0.071, 'epoch': 2.0}


                                               
100%|██████████| 60/60 [09:41<00:00,  9.70s/it]

{'eval_loss': 2.8674826622009277, 'eval_chrf_score': 0.03242280617316204, 'eval_runtime': 22.967, 'eval_samples_per_second': 2.308, 'eval_steps_per_second': 0.305, 'epoch': 3.0}
{'train_runtime': 582.1429, 'train_samples_per_second': 0.809, 'train_steps_per_second': 0.103, 'train_loss': 1.4203641255696615, 'epoch': 3.0}


TrainOutput(global_step=60, training_loss=1.4203641255696615, metrics={'train_runtime': 582.1429, 'train_samples_per_second': 0.809, 'train_steps_per_second': 0.103, 'train_loss': 1.4203641255696615, 'epoch': 3.0})